#Mounting the Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import Statements

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers import LSTM, GRU
from keras.layers import Convolution1D, MaxPooling1D
from keras.regularizers import l2
from keras.layers.core import Lambda, Flatten, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Bidirectional
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from sklearn.metrics import roc_auc_score
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.initializers import glorot_uniform
from keras.layers import Input, Dense, Flatten, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract, Add, Conv2D
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import keras.backend as K
from keras.models import model_from_json
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import re

#Loading pure English dataset

In [ ]:
data_source = '/content/drive/MyDrive/Data/Tweets.csv'
train_df = pd.read_csv(data_source, header=None)
train_df = train_df.iloc[1:]
train_df.head()
# len(train_df)

,0,1
1,1,@VirginAmerica What @dhepburn said.
2,2,@VirginAmerica plus you've added commercials t...
3,1,@VirginAmerica I didn't today... Must mean I n...
4,0,@VirginAmerica it's really aggressive to blast...
5,0,@VirginAmerica and it's a really big bad thing...


#Preprocessing the loaded dataset

In [ ]:
train_df[1] = train_df[1].str.lower()
train_df[1] = train_df[1].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
train_df[1]= train_df[1].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
train_df[1] = train_df[1] .apply(lambda x: re.sub(r'{link}', '', x))
train_df[1]= train_df[1].apply(lambda x: re.sub(r"\[video\]", '', x))
train_df[1]=train_df[1] .apply(lambda x: re.sub(r'&[a-z]+;', '', x))
train_df[1] = train_df[1].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))
l = ["virginamerica","united","jetblue","southwestair","usairways","americanair"]


In [ ]:
texts = train_df[1].values # texts contrain all setences
print(len(texts))
print(texts[:2])
for i in range(len(texts)):
  for x in l:
    if x in texts[i]:
      texts[i] = texts[i].replace(x,"")
print(len(texts))
print(texts[:2])

14640
['virginamerica what dhepburn said'
 "virginamerica plus you've added commercials to the experience tacky"]
14640
[' what dhepburn said'
 " plus you've added commercials to the experience tacky"]


#Loading the Hinglish Dataset

In [ ]:
data_source2 = '/content/drive/MyDrive/Data/Hindi_english_dataset.csv'
train_df2 = pd.read_csv(data_source2, header=None)
train_df2 = train_df2.iloc[1:]
train_df2.head()
len(train_df2)

6357

#Preprocessing the loaded dataset

##Regex for removing unnecessary text from the dataset

In [ ]:
train_df2[0] = train_df2[0].str.lower()
train_df2[0] = train_df2[0].apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
train_df2[0]= train_df2[0].apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))
train_df2[0] = train_df2[0] .apply(lambda x: re.sub(r'{link}', '', x))
train_df2[0]= train_df2[0].apply(lambda x: re.sub(r"\[video\]", '', x))
train_df2[0]=train_df2[0] .apply(lambda x: re.sub(r'&[a-z]+;', '', x))
train_df2[0] = train_df2[0].apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))


In [ ]:
texts2 = train_df2[0].values # texts contrain all setences
#texts = [s.lower() for s in texts] # convert to lower case 
print(len(texts2))
print(texts2[:2])
textlab =  train_df2[1].values
for i in range(len(textlab)):
  textlab[i]=int(textlab[i])
train_df2[2]=textlab
train_df2

6357
['jab tak kohli captain se aur shastri coach se dismissed nhi honge tab tak koi big tournament jeetenge hi nhi'
 'really i  agree to you viru paaji ab ravi shastri ko istifa dena chaiye']


,0,1,2
1,jab tak kohli captain se aur shastri coach se ...,0,0
2,really i agree to you viru paaji ab ravi shas...,1,1
3,aur ke players ki mentality aur techniques s...,1,1
4,kewal ravi shashtri ko bhagao indian team phir...,0,0
5,jab rohitsharma run bana raha tha to ye godi ...,0,0
...,...,...,...
6353,hindu muslim mien kon sa payar ha jo akbar bha...,1,1
6354,hindu muslim ka asli dange toye mc modi kar ra...,1,1
6355,are randi wali mother chood modiyaa kee khil...,0,0
6356,modi ji ne jo aatang danga cheating karke indi...,1,1


##Selecting positive and negative sentiment sentences

###From pure English dataset

In [ ]:
positive_Eng = list(train_df[train_df[0] == '2'][1])
negative_Eng = list(train_df[train_df[0] == '0'][1])


###From Hinglish dataset

In [ ]:
positive_HiEng = list(train_df2[train_df2[2] == 2][0])
negative_HiEng = list(train_df2[train_df2[2] == 0][0])
# len(train_df2[train_df2[2] == 1][0])



In [ ]:
print(negative_HiEng[:2])
l=list(train_df[0])
type(l[0])

['jab tak kohli captain se aur shastri coach se dismissed nhi honge tab tak koi big tournament jeetenge hi nhi', 'kewal ravi shashtri ko bhagao indian team phir se bare tournament jeetne lagegi dhanyawad ']


str

##Selecting the same number of sentences for each sentiment from both the datasets

In [ ]:
Pos_eng_6k= positive_Eng[:2000]
neg_eng_6k = negative_Eng[:2000]
Pos_hieng_6k = positive_HiEng[:2000]
neg_hieng_6k = negative_HiEng[:2000]
len(negative_HiEng)

1637

Text1: Anchor sentence from pure english <br>
Text2: Pairing a permutation of sentiments with the anchor <br>
Label: 0: If both belong to opposite sentiment, 1: If both belong to the same sentiment
      

In [ ]:
new_df = pd.DataFrame(columns=['text1', 'text2', 'label'])

In [ ]:
for data in Pos_eng_6k:   #Taking anchor as Positive
  data1 = data
  data2 = random.choice(Pos_hieng_6k)
  data3 = random.choice(neg_hieng_6k)

  new_df.loc[len(new_df)] = [data1, data2, 1]
  new_df.loc[len(new_df)] = [data1, data3, 0]
new_df[:10]

,text1,text2,label
0,plus you've added commercials to the experien...,modi g ke jajbe ko koti koti naman,1
1,plus you've added commercials to the experien...,do chutiya ek saath great moment,0
2,yes nearly every time i fly vx this ear worm ...,two ligands jay hind,1
3,yes nearly every time i fly vx this ear worm ...,chutiya bna rahey ho kya bhai:p boley to lucky...,0
4,well i didn'tbut now i do :-d,aaj dil khush hua iska sabse bada faisla mil gya,1
5,well i didn'tbut now i do :-d,kaheytey ho ki muslim fans pic nahi dekhey to ...,0
6,it was amazing and arrived an hour early you'...,jaanma main bol rahi hu kitum mere twits dekho :/,1
7,it was amazing and arrived an hour early you'...,jb over me runs chahiye or dhoni balls khel...,0
8,i pretty graphics so much better than minima...,isse pta chalta hai caa nrc or population cont...,1
9,i pretty graphics so much better than minima...,nahi to ye mat soch ke tu fanna nahi hoga ye k...,0


In [ ]:
for data in neg_eng_6k:   #Taking anchor as Negative and appending at aend of df2
  data1 = data
  data2 = random.choice(neg_hieng_6k)
  data3 = random.choice(Pos_hieng_6k)

  new_df.loc[len(new_df)] = [data1, data2, 1]
  new_df.loc[len(new_df)] = [data1, data3, 0]


In [ ]:
new_df.shape

(8000, 3)

In [ ]:
display(new_df.sample(5))

,text1,text2,label
7135,we are delayed in san pedro belize we are sch...,wah sir g ab desh sahi hatho me gya h apko dek...,0
2425,thanks do yall expect to be operational tomor...,aur sunlo humko reply krne waalon hum to usko ...,0
6603,i bought a ticket with a price that was publi...,superb salman bhai,0
7091,stuck in dc trying to get to denver the engin...,yess bhaaijaan hum aapke sath hai hamesha kuch...,0
5344,is the worst worst reservation policies worst...,finally nirbhaya got justice rip nirbhayauun d...,1


##Splitting the dataset into training, testing and validation sets

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(new_df[['text1', 'text2']], new_df['label'], test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_temp[['text1', 'text2']], y_temp, test_size=0.2, random_state=0)

In [ ]:
new_df['text'] = new_df[['text1', 'text2']].apply(lambda x: str(x[0])+" "+str(x[1]), axis=1)
new_df

,text1,text2,label,text
0,plus you've added commercials to the experien...,modi g ke jajbe ko koti koti naman,1,plus you've added commercials to the experien...
1,plus you've added commercials to the experien...,do chutiya ek saath great moment,0,plus you've added commercials to the experien...
2,yes nearly every time i fly vx this ear worm ...,two ligands jay hind,1,yes nearly every time i fly vx this ear worm ...
3,yes nearly every time i fly vx this ear worm ...,chutiya bna rahey ho kya bhai:p boley to lucky...,0,yes nearly every time i fly vx this ear worm ...
4,well i didn'tbut now i do :-d,aaj dil khush hua iska sabse bada faisla mil gya,1,well i didn'tbut now i do :-d aaj dil khush h...
...,...,...,...,...
7995,your airline is the biggest joke of an operat...,you r great salman siraap actor baad me ho peh...,0,your airline is the biggest joke of an operat...
7996,what is going on with baggage claim in newark...,in suwro ko \nkonkon bich choraye pr latkana ...,1,what is going on with baggage claim in newark...
7997,what is going on with baggage claim in newark...,salman ji aapne iss bar koi song nahi gaya muj...,0,what is going on with baggage claim in newark...
7998,usually your lounge staff are fantastic excep...,ek madhar chod ravish kumar ab ise justify krn...,1,usually your lounge staff are fantastic excep...


In [ ]:
embed_text = new_df['text'].values # texts contrain all setences
embed_text = [s.lower() for s in embed_text] # convert to lower case 
print(len(embed_text))
print(embed_text[:2])


8000
[" plus you've added commercials to the experience tacky modi g ke jajbe ko koti koti naman", " plus you've added commercials to the experience tacky do chutiya ek saath great moment"]


#Creating the vocabulary from the join sentences

In [ ]:
# import string
import re
for i in range(len(embed_text)):
  embed_text[i] = re.sub(r'[^a-z0-9\s]', '', embed_text[i])


In [ ]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(embed_text)
tk.word_index

{'\n': 29,
 ' ': 2,
 'UNK': 1,
 'a': 3,
 'b': 19,
 'c': 20,
 'd': 14,
 'e': 4,
 'f': 22,
 'g': 18,
 'h': 8,
 'i': 5,
 'j': 25,
 'k': 15,
 'l': 12,
 'm': 16,
 'n': 9,
 'o': 7,
 'p': 21,
 'q': 28,
 'r': 10,
 's': 11,
 't': 6,
 'u': 13,
 'v': 24,
 'w': 23,
 'x': 27,
 'y': 17,
 'z': 26}

In [ ]:
alphabet="abcdefghijklmnopqrstuvwxyz0123456789"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
char_dict

{'0': 27,
 '1': 28,
 '2': 29,
 '3': 30,
 '4': 31,
 '5': 32,
 '6': 33,
 '7': 34,
 '8': 35,
 '9': 36,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

In [ ]:
tk.word_index

{'0': 27,
 '1': 28,
 '2': 29,
 '3': 30,
 '4': 31,
 '5': 32,
 '6': 33,
 '7': 34,
 '8': 35,
 '9': 36,
 'UNK': 37,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

#Data Preparation

In [ ]:
X_train['text1'] = X_train['text1'].astype(str)
X_train['text2'] = X_train['text2'].astype(str)
X_val['text1'] = X_val['text1'].astype(str)
X_val['text2'] = X_val['text2'].astype(str)
X_test['text2'] = X_test['text2'].astype(str)
X_test['text1'] = X_test['text1'].astype(str)

train_q1_seq = tk.texts_to_sequences(X_train['text1'].values)
train_q2_seq = tk.texts_to_sequences(X_train['text2'].values)
val_q1_seq = tk.texts_to_sequences(X_val['text1'].values)
val_q2_seq = tk.texts_to_sequences(X_val['text2'].values)
test_q1_seq = tk.texts_to_sequences(X_test['text1'].values)
test_q2_seq = tk.texts_to_sequences(X_test['text2'].values)

max_len = 200
train_q1_seq = pad_sequences(train_q1_seq, maxlen=max_len, padding='post')
train_q2_seq = pad_sequences(train_q2_seq, maxlen=max_len, padding='post')
test_q1_seq = pad_sequences(test_q1_seq, maxlen=max_len, padding='post')
test_q2_seq = pad_sequences(test_q2_seq, maxlen=max_len, padding='post')
val_q1_seq = pad_sequences(val_q1_seq, maxlen=max_len, padding='post')
val_q2_seq = pad_sequences(val_q2_seq, maxlen=max_len, padding='post')

#Forming Embedding Layer

In [ ]:
vocab_size = len(tk.word_index)
embedding_layer = Embedding(vocab_size + 1, 128, 
                            input_length=train_q1_seq.shape[1])

Def

#Defining functions for Euclidean distance and contrastive loss

In [ ]:
def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    # return the computed contrastive loss to the calling function
    return loss

#Defining function for our model

##Model 1.0: LSTM + 2 dense layers

In [ ]:
def build_network():
  model = Sequential()
  model.add(Embedding(input_dim=38, output_dim=128, input_length=200))
  model.add(Conv1D(filters=128,
            kernel_size=3,
            padding='valid',
            activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  #network.add(Embedding(name="synopsis_embedd",input_dim =len(tk.word_index)+1, 
                       #output_dim=len(embeddings_index['no']),weights=[embedding_matrix], 
                       #input_length=train_q1_seq.shape[1],trainable=False))
  model.add(LSTM(64,return_sequences=True, activation="relu"))
  model.add(Flatten())
  model.add(Dense(128, activation='relu',
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer='he_uniform'))
  
  model.add(Dense(2, activation=None,
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer='he_uniform'))
  
  #Force the encoding to live on the d-dimentional hypershpere
  model.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))

  return model

##Model 2.0: Bi-LSTM + 1 dense layer

In [ ]:
def build_network_2():
  model = Sequential()
  model.add(Embedding(input_dim=38, output_dim=128, input_length=200))
  model.add(Conv1D(filters=128,
            kernel_size=3,
            padding='valid',
            activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  #network.add(Embedding(name="synopsis_embedd",input_dim =len(tk.word_index)+1, 
                       #output_dim=len(embeddings_index['no']),weights=[embedding_matrix], 
                       #input_length=train_q1_seq.shape[1],trainable=False))
  model.add(Bidirectional(LSTM(64,return_sequences=True, activation="relu")))
  model.add(Flatten())  
  model.add(Dense(2, activation=None,
                  kernel_regularizer=l2(1e-3),
                  kernel_initializer='he_uniform'))
  
  #Force the encoding to live on the d-dimentional hypershpere
  model.add(Lambda(lambda x: K.l2_normalize(x,axis=-1)))

  return model

##Model 3.0: Two LSTMs (Joshi implementation)

In [ ]:
def build_network_3():
  model = Sequential()
  model.add(Embedding(input_dim=38, output_dim=128, input_length=200))
  model.add(Conv1D(filters=128,
            kernel_size=3,
            padding='valid',
            activation='relu'))
  model.add(MaxPooling1D(pool_size=2))
  model.add(LSTM(128, recurrent_dropout=0.2, dropout=0.2, return_sequences=True))
  model.add(LSTM(128, recurrent_dropout=0.2, dropout=0.2, return_sequences=False))
  model.add(Dense(2, activation='softmax'))

  return model

#Taking inputs from the dataset and passing to the model for training

In [ ]:
input_1 = Input(shape=(train_q1_seq.shape[1],))
input_2 = Input(shape=(train_q2_seq.shape[1],))

test_input_1 = Input(shape=(test_q1_seq.shape[1],))
test_input_2 = Input(shape=(test_q2_seq.shape[1],))

network = build_network()

encoded_input_1 = network(input_1)
encoded_input_2 = network(input_2)



##Starting the training of the model

In [ ]:
distance = Lambda(euclidean_distance)([encoded_input_1, encoded_input_2])
# Connect the inputs with the outputs
model = Model([input_1, input_2], distance)
model.compile(loss=contrastive_loss, optimizer=Adam(0.001), metrics=['accuracy'])
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')
y_test = np.asarray(y_test).astype('float32')
model.fit([train_q1_seq,train_q2_seq],y_train.reshape(-1,1), epochs = 8, 
          batch_size=64,validation_data=([val_q1_seq, val_q2_seq],y_val.reshape(-1,1)))

Epoch 1/8
80/80 [==============================] - 36s 415ms/step - loss: 0.4911 - accuracy: 0.4756 - val_loss: 0.4235 - val_accuracy: 0.4375
Epoch 2/8
80/80 [==============================] - 33s 411ms/step - loss: 0.4202 - accuracy: 0.4678 - val_loss: 0.3903 - val_accuracy: 0.4430
Epoch 3/8
80/80 [==============================] - 33s 411ms/step - loss: 0.3920 - accuracy: 0.4695 - val_loss: 0.3662 - val_accuracy: 0.4406
Epoch 4/8
80/80 [==============================] - 33s 411ms/step - loss: 0.3704 - accuracy: 0.4656 - val_loss: 0.3602 - val_accuracy: 0.4711
Epoch 5/8
80/80 [==============================] - 33s 409ms/step - loss: 0.3571 - accuracy: 0.4656 - val_loss: 0.3418 - val_accuracy: 0.4523
Epoch 6/8
80/80 [==============================] - 33s 411ms/step - loss: 0.3449 - accuracy: 0.4656 - val_loss: 0.3359 - val_accuracy: 0.4578
Epoch 7/8
80/80 [==============================] - 33s 409ms/step - loss: 0.3337 - accuracy: 0.4584 - val_loss: 0.3596 - val_accuracy: 0.4633
Epoch 

#Saving the model

In [ ]:
# Save model for further use
# serialize model to JSON
model_json = model.to_json()
with open("siamesemodel-contrastive-loss.json", "w") as json_file:
    json_file.write(model_json)
#serialize weights to HDF5
model.save_weights("siamesemodel-contrastive-loss.h5")



#Loading the same model

In [ ]:
json_file = open('siamesemodel-contrastive-loss.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("siamesemodel-contrastive-loss.h5")
# loaded_model.compile(loss=contrastive_loss, optimizer=Adam(0.001), metrics=['accuracy'])
# loaded_model.fit([train_q1_seq,train_q2_seq],y_train.reshape(-1,1), epochs = 5, 
# batch_size=64,validation_data=([val_q1_seq, val_q2_seq],y_val.reshape(-1,1)))

#Prediction phase

##Predicting for a positive using a positive assistant

In [ ]:
prediction_data = "Good deed modi bhai ji......hme hmara desh fir se golden sparrow bnana h....wo spna sch hoga...jiske liye god n apko select kiya h...thnku god"
prediction_vector = tk.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = "masha allah good....yeh hain meri hindi"
assistant_vector = tk.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)
loaded_model.predict([prediction_vector, assistant_vector])

array([[0.92679805]], dtype=float32)

##Predicting for a negative using a positive assistant

In [ ]:
prediction_data = "Kyo modi ji bihar ke yua berojgar hi rhenge. Bihar ki gandi rajniti me sare yua piss rhe h"
prediction_vector = tk.texts_to_sequences([prediction_data])
prediction_vector = pad_sequences(prediction_vector,maxlen=200)

assistant_data = "Amazing respect for modi even  years back salute to you Amit sir aap logo ki jai"
assistant_vector = tk.texts_to_sequences([assistant_data])
assistant_vector = pad_sequences(assistant_vector,maxlen=200)

loaded_model.predict([prediction_vector, assistant_vector])

array([[0.04976079]], dtype=float32)

#Evalutaing the model

In [ ]:
results = model.evaluate([test_q1_seq,test_q2_seq], y_test.reshape(-1,1), batch_size=128)
print("test loss, test acc:", results)


13/13 [==============================] - 3s 192ms/step - loss: 0.3395 - accuracy: 0.4769
test loss, test acc: [0.33949315547943115, 0.4768750071525574]
